# Using MoXing to recognize handwritten number 

This content mainly introduces how to use MoXing to implement training and testing applications of handwritten digital images.

## 1. Preparing Data

Download the MNIST data set from the mnist_data object in the mnist bucket of obs and upload it to the private OBS bucket. You can fidn data here:

https://modelarts-labs.oss.eu-west-0.prod-cloud-ocb.orange-business.com/dataset-mnist/zip-data/t10k-images-idx3-ubyte.gz

https://modelarts-labs.oss.eu-west-0.prod-cloud-ocb.orange-business.com/dataset-mnist/zip-data/t10k-labels-idx1-ubyte.gz

https://modelarts-labs.oss.eu-west-0.prod-cloud-ocb.orange-business.com/dataset-mnist/zip-data/train-images-idx3-ubyte.gz

https://modelarts-labs.oss.eu-west-0.prod-cloud-ocb.orange-business.com/dataset-mnist/zip-data/train-labels-idx1-ubyte.gz

### 1.1 Download the MNIST data set, the data set file description is as follows:

t10k-images-idx3-ubyte.gz: The verification set contains 10,000 samples in total.

t10k-labels-idx1-ubyte.gz: Validation set labels, including category labels for 10,000 samples.

train-images-idx3-ubyte.gz: The training set contains a total of 60,000 samples.

train-labels-idx1-ubyte.gz: training set labels, including category labels for 60,000 samples.

### 1.2 The .gz data does not need to be decompressed and uploaded to Your Flexible Engine OBS bucket separately. The data path will be set to data_url

## 2. Training The Model

Load MoXing's tensorflow module moxing.tensorflow by import

In [3]:
import moxing.tensorflow as mox
import os

INFO:root:Using MoXing-v1.15.1-3fc51aac
INFO:root:Using OBS-Python-SDK-3.1.2
INFO:tensorflow:Using TensorFlow-b'v1.13.1-0-g6612da8951'


Set data_url and train_url according to data storage and data output

In [4]:
data_url = 's3://modelarts-labs/dataset-mnist/zip-data/' 

In [6]:
# https://github.com/huaweicloud/ModelArts-Lab/blob/master/official_examples/Using_Notebook_to_Create_a_MNIST_Dataset_Recognition_Application/code/mnist_example.ipynb
train_url = './cache/log/'
if not mox.file.exists(data_url):
    raise ValueError('Plese verify your data url!')
if mox.file.exists(train_url):
    mox.file.remove(train_url,recursive=True)
mox.file.make_dirs(train_url)

The data can be copied locally through mox, which can speed up training. The operation is as follows:

In [7]:
local_url = './cache/local_data/'
if mox.file.exists(local_url):
    mox.file.remove(local_url,recursive=True)
os.makedirs(local_url)

In [8]:
mox.file.copy_parallel(data_url, local_url)
data_url = local_url
os.listdir(data_url)

['t10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz',
 'train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz']

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from __future__ import print_function
from __future__ import unicode_literals

In [10]:
tf.flags.DEFINE_string('data_url', None, 'Dir of dataset')
tf.flags.DEFINE_string('train_url', None, 'Train Url')

flags = tf.flags.FLAGS

filenames = ['train-images-idx3-ubyte.gz','train-labels-idx1-ubyte.gz','t10k-images-idx3-ubyte.gz',
             't10k-labels-idx1-ubyte.gz']

for filename in filenames:
  filepath = os.path.join(data_url, filename)
  if not mox.file.exists(filepath):
    raise ValueError('MNIST dataset file %s not found in %s' % (filepath, local_url))

In [11]:
def main(*args):
  flags.data_url = data_url
  flags.train_url = train_url
  mnist = input_data.read_data_sets(flags.data_url, one_hot=True)
        

  # define the input dataset, return image and label
  def input_fn(run_mode, **kwargs):
    def gen():
      while True:
        yield mnist.train.next_batch(50)
    ds = tf.data.Dataset.from_generator(
        gen, output_types=(tf.float32, tf.int64),
        output_shapes=(tf.TensorShape([None, 784]), tf.TensorShape([None, 10])))
    return ds.make_one_shot_iterator().get_next()


  # define the model for training or evaling.
  def model_fn(inputs, run_mode, **kwargs):
    x, y_ = inputs
    W = tf.get_variable(name='W', initializer=tf.zeros([784, 10]))
    b = tf.get_variable(name='b', initializer=tf.zeros([10]))
    y = tf.matmul(x, W) + b
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    predictions = tf.argmax(y, 1)
    correct_predictions = tf.equal(predictions, tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    export_spec = mox.ExportSpec(inputs_dict={'images': x}, outputs_dict={'predictions': predictions}, version='model')
    return mox.ModelSpec(loss=cross_entropy, log_info={'loss': cross_entropy, 'accuracy': accuracy},
                         export_spec=export_spec)


  mox.run(input_fn=input_fn,
          model_fn=model_fn,
          optimizer_fn=mox.get_optimizer_fn('sgd', learning_rate=0.01),
          run_mode=mox.ModeKeys.TRAIN,
          batch_size=50,
          auto_batch=False,
          log_dir=flags.train_url,
          max_number_of_steps=1000,
          log_every_n_steps=10,
          export_model=mox.ExportKeys.TF_SERVING)

if __name__ == '__main__':
  try:
      tf.app.run(main=main)
  except SystemExit:
      pass

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting ./cache/local_data/train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use tf.one_hot on tensors.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ./cache/local_data/train-labels-idx1-ubyte.gz
Extracting ./cache/local_data/t10k-images-idx3-ubyte.gz
Extracting ./cache/local_data/t10k-labels-idx1-ubyte.gz


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:ten

## 3. Inference

In [12]:
src_path = 's3://modelarts-labs/dataset-mnist/data/test.jpg'

In [13]:
if not mox.file.exists(src_path):
    raise ValueError('Plese verify your src_path!')
dst_path =  './cache/test.jpg'
mox.file.copy(src_path,dst_path)

In [14]:
image_path = './cache/test.jpg'            # 指定图片位置
checkpoint_url = './cache/log/'         # 指定checkpoint位置，即上一步训练指定的路径的位置。
print(mox.file.exists(image_path))

True


In [15]:
import moxing.tensorflow as mox
import os
import tensorflow as tf
from __future__ import print_function
from __future__ import unicode_literals

In [17]:
def predict(*args):
  def input_fn(run_mode, **kwargs):
    image = tf.read_file(image_path)
    img = tf.image.decode_jpeg(image, channels=1)
    img = tf.image.resize_images(img, [28, 28], 0)
    img = tf.reshape(img, [784])
    return img

  def model_fn(inputs, run_mode, **kwargs):
    x = inputs
    W1 = tf.get_variable(name='W', initializer=tf.zeros([784, 10]))
    b1 = tf.get_variable(name='b', initializer=tf.zeros([10]))
    y = tf.matmul(x, W1) + b1
    predictions = tf.argmax(y, 1)
    return mox.ModelSpec(output_info={'predict': predictions})

  def output_fn(outputs):
    for output in outputs:
      result = output['predict']
      print("The result：",result)

  mox.run(input_fn=input_fn,
          model_fn=model_fn,
          output_fn=output_fn,
          run_mode=mox.ModeKeys.PREDICT,
          batch_size=1,
          auto_batch=False,
          max_number_of_steps=1,
          output_every_n_steps=1,
          checkpoint_path=checkpoint_url)
if __name__ == '__main__':
  try:
      tf.app.run(main=predict)
  except SystemExit:
      pass

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cache/log/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:	[1 examples]


The result： [7]
